In [6]:
# Ряд сопротивлений Е24
E24 = (1.0, 1.1, 1.2, 1.3, 1.5, 1.6, 1.8, 2.0, 2.2, 2.4, 2.7, 3.0, 3.3, 3.6, 3.9, 4.3, 4.7, 5.1, 5.6, 6.2, 6.8, 7.5, 8.2, 9.1)

def format_r(value=0.0):
    ''' Форматирование значения сопротивления
    Например: 2.2 кОм
    '''
    for r_factor, text_factor in ((1e6, u'МОм'), (1e3, u'кОм'), (1.0, u'Ом')):
        if divmod(value, r_factor)[0] > 0.0:
            return u'{0} {1}'.format(value / r_factor,  text_factor)

def Gain_noninv_calc(r1=0.0, r2=0.0):
    ''' Возвращает Кус неинвертирующего усилителя для двух сопротивлений '''
    return (r1 + r2) / r2

def Gain_inv_calc(r1=0.0, r2=0.0):
    ''' Возвращает Кус инвертирующего усилителя для двух сопротивлений '''
    return r2 / r1

def Kdiv_calc(r1=0.0, r2=0.0):
    ''' Возвращает Кдел резистивного делителя напряжения для двух сопротивлений '''
    return r2 / (r1 + r2)

def Kdiv_calc2(r1=0.0, r2=0.0):
    ''' Возвращает Кдел резистивного делителя напряжения для двух сопротивлений '''
    return r2 / (r1 + r2)

def Rparallel_calc(r1=0.0, r2=0.0):
    ''' Возвращает сопротивление двух параллельно соединенённых резисторов '''
    return (r1 * r2) / (r1 + r2)

def get_rvalues(r_series=E24, r_range=(1.0, 10e6)):
    ''' Возвращает список пар сопротивлений делителя в соответствии с фильтром
    r_series -- Ряд сопротивлений. Например: E24
    r_range  -- Фильтр (диапазон) сопротивлений, включительно. Например: 1..100 кОм (1e3, 1e5)
    '''
    ret = []
    for r_factor in (1.0, 10.0, 1e2, 1e3, 1e4, 1e5, 1e6):
        for r_base in r_series:
            r = r_base * r_factor
            if r >= min(r_range) and r <= max(r_range):
                ret.append(r)
    return ret

def get_rdiv(r_values=E24, k_specified=1.0, k_tlr=0.1, calc_function=Kdiv_calc):
    ''' Возврщает список пар сопротивлений делителя в соответствии с допуском
    r_values      -- Ряд сопротивлений. Например: E24
    k_specified   -- Заданный коэффициент делителя
    k_tlr         -- Допуск, %
    calc_function -- Функция расчёта коэффициента делителя для двух сопротивлений
    '''
    k_tlr /= 100
    ret = []
    for r1 in r_values:
        for r2 in r_values:
            k_div = calc_function(r1, r2)
            if k_div >= k_specified * (1 - k_tlr) and k_div <= k_specified * (1 + k_tlr):
                ret.append((r1, r2))
    return ret

Kdiv_precision = 3
import math
def print_result(print_values_as_list=False):
    def rowValues(r1, r2):
        ''' Возвращает вариации коэф. деления для предельных отклонений R2 '''
        return [Kdiv_calc_function(r1, r2 * (1 - R_tlr / 100)), Kdiv_calc_function(r1, r2), Kdiv_calc_function(r1, r2 * (1 + R_tlr / 100))]
    def list_variation_max(value=0.0, variations=[]):
        max_val_abs = 0.0
        max_val = 0.0
        for i in range(len(variations)):
            if max_val_abs < math.fabs(variations[i] - value):
                max_val = variations[i] - value
                max_val_abs = math.fabs(max_val)
        return max_val
    print('Заданное {}: {:^8} ±{}%\t\tДопуск резисторов: {}%'.format(Kdiv_description, Kdiv, Kdiv_tlr, R_tlr))
    print('{:>8} {:^12}{:^12}{:^12}{:^12}{:^12}{:^15} '.format(u'Погрешность', Kdiv_description, 'R1', 'R2', Kdiv_description + '.min', Kdiv_description + '.max', u'Погрешность.max'))
#    kdiv_print_format = u'{{:>8.3f}}%{{:>12.{0}f}}{{:>12}}{{:>12}}{{:>12.{0}f}}{{:>12.{0}f}}{{:>15.3f}}%\t\t[{{:.{0}f}}, {{:.{0}f}}, {{:.{0}f}}]'.format(Kdiv_precision)
    kdiv_print_format = '{{:>8.3f}}%{{:>12.{0}f}}{{:>12}}{{:>12}}{{:>12.{0}f}}{{:>12.{0}f}}{{:>15.3f}}%'.format(Kdiv_precision)
    if print_values_as_list:
        kdiv_print_format += u'\t\t[{{:.{0}f}}, {{:.{0}f}}, {{:.{0}f}}]'.format(Kdiv_precision)
    for r in Rdiv:
        k_variations = rowValues(r[0] * (1 - R_tlr / 100), r[1]) + rowValues(r[0], r[1]) + rowValues(r[0] * (1 + R_tlr / 100), r[1])
        k_div = Kdiv_calc_function(r[0], r[1])
        print(kdiv_print_format.format(((k_div - Kdiv) / Kdiv) * 100, k_div, format_r(r[0]), format_r(r[1]), min(k_variations), max(k_variations), (list_variation_max(Kdiv, k_variations) / Kdiv) * 100, min(k_variations), Kdiv_calc_function(r[0], r[1]), max(k_variations)))


In [7]:
# подобрать пару резисторов для инвертирующего усилителя
#Kdiv = .946 ; Kdiv_tlr = 0.1 ; R_tlr = 1.0 ; Kdiv_description = u'Кдел' ; Kdiv_precision = 4 ; Kdiv_calc_function = Kdiv_calc
#Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr, Kdiv_calc_function)
#print_result()
#print

# подобрать пары резисторов для неинвертирующего усилителя

from IPython.display import Image, display
display(Image(filename='img/current_sensor.png'))

Kdiv_description = u'1/Кус' ; Kdiv_precision = 3 ; Kdiv_calc_function = Kdiv_calc

Uout_max = 2.5 # V

# подобрать пару резисторов для неинвертирующего усилителя
print 'Uref=Vdda\nUref_int=1.224В\nUout max={} В\n'.format(Uout_max)
Ush = 0.165
print 'HL шунт\nImax=0.75A Rш=0.22Ω Uш={}В'.format(Ush)
Kdiv = Ush / Uout_max ; Kdiv_tlr = 2.0 ; R_tlr = 1.0
Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr)
print_result()
print

# подобрать пару резисторов для неинвертирующего усилителя
Ush = 0.132
print 'BAT шунт\nImax=1.2A Rш=0.11Ω Uш={}В'.format(Ush)
Kdiv = 0.132 / Uout_max ; Kdiv_tlr = 0.5 ; R_tlr = 1.0
Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr)
print_result()
print

# подобрать пару резисторов для инвертирующего усилителя
Kdiv_calc_function = Gain_inv_calc
Ush = 0.125
print 'USB шунт\nImax=2.5A Rш=0.05Ω Uш={}В'.format(Ush)
Kdiv = 0.125 / Uout_max ; Kdiv_tlr = 2.0 ; R_tlr = 1.0
Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr)
print_result()
print

# подобрать пару резисторов для делителя напряжения
#Kdiv = 11.13 ; Kdiv_tlr = 1.0 ; R_tlr = 1.0 ; Kdiv_description = u'Кус' ; Kdiv_precision = 3 ; Kdiv_calc_function = Gain_calc
#Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr, Kdiv_calc_function)
#print_result()
#print

# подобрать пару резисторов для параллельного соединения
#Kdiv = 17e3 ; Kdiv_tlr = 1.0 ; R_tlr = 1.0 ; Kdiv_description = u'Rпар' ; Kdiv_calc_function = Rparallel_calc
#Rdiv = get_rdiv(get_rvalues(E24, (1e3, 1e5)), Kdiv, Kdiv_tlr, Kdiv_calc_function)
#print_result()
#print

SyntaxError: invalid syntax (<ipython-input-7-10f7f0a6758f>, line 17)